<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [6]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
import requests

# URL of the Falcon9 Launch Wikipedia page
static_url = "https://en.wikipedia.org/wiki/Falcon_9"

# Send HTTP GET request to the URL
response = requests.get(static_url)

# Verify the request was successful (status code 200)
if response.status_code == 200:
    print("Successfully retrieved the Falcon9 Launch Wiki page!")
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

# The HTML content is stored in response.text
# Example: html_content = response.text

Successfully retrieved the Falcon9 Launch Wiki page!


Create a `BeautifulSoup` object from the HTML `response`


In [8]:
import requests
from bs4 import BeautifulSoup

# Step 1: Fetch the webpage
url = "https://en.wikipedia.org/wiki/Falcon_9"
response = requests.get(url)

# Step 2: Create BeautifulSoup object
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    print("BeautifulSoup object created successfully!")
    
    # Example: Find the page title
    title = soup.title.string
    print(f"Page Title: {title}")
else:
    print(f"Request failed with status code: {response.status_code}")

BeautifulSoup object created successfully!
Page Title: Falcon 9 - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [11]:
# Print the page title to verify the BeautifulSoup object
print(soup.title.string)


Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [12]:
# TASK 2: Extract all column names from the HTML table header

# Find all tables on the page
html_tables = soup.find_all('table')

# Select the first table (or the one you need based on class or index)
first_table = html_tables[0]

# Extract header cells (<th>) from the first row of the table
header_cells = first_table.find('tr').find_all('th')

# Get the text for each header and strip whitespace
column_names = [th.get_text(strip=True) for th in header_cells]

print(column_names)



[]


Starting from the third table is our target table contains the actual launch records.


In [13]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin engines
</td>
<td>9 sea level
</td></tr>
<tr>
<td>Propellant
</td>
<td>LOX / RP-1

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [25]:
import requests
from bs4 import BeautifulSoup
import re

# Используем «замороженный» снимок страницы с oldid
static_url = (
    "https://en.wikipedia.org/w/index.php?"
    "title=List_of_Falcon_9_and_Falcon_Heavy_launches&"
    "oldid=1027686922"
)

# 1) Получаем HTML
response = requests.get(static_url)
soup = BeautifulSoup(response.text, 'html.parser')

# 2) Находим таблицу запусков по её классу
launch_table = soup.find(
    'table',
    class_="wikitable plainrowheaders collapsible"
)
if not launch_table:
    raise ValueError("Таблица запусков не найдена на статическом снимке!")

# 3) Функция очистки текста заголовка <th>
def extract_column_from_header(th_tag):
    text = th_tag.get_text(separator=" ", strip=True)
    # Убираем [1], [a] и т.п.
    return re.sub(r"\[\w+\]", "", text) or None

# 4) Извлекаем <th> из <thead>
thead = launch_table.find('thead')
header_row = thead.find('tr') if thead else launch_table.find('tr')

column_names = []
for th in header_row.find_all('th'):
    name = extract_column_from_header(th)
    if name:
        column_names.append(name)

print(column_names)


['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


Check the extracted column names


In [26]:
print(column_names)

['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [28]:
# Инициализация словаря с ключами из column_names
launch_dict = dict.fromkeys(column_names)

# Удаляем неактуальный ключ (точное совпадение названия из списка column_names)
del launch_dict['Date and time ( UTC )']

# Переопределяем каждый ключ списком
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload [ c ]': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],    # вместо 'Version, Booster [ b ]'
    'Booster landing': [],
    'Date': [],
    'Time': []
}

# Проверка
print(launch_dict.keys())


dict_keys(['Flight No.', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'])


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [31]:
# TASK 3: Заполнение launch_dict из таблицы запусков

# Предполагаем, что launch_dict уже инициализирован:
launch_dict = {
    'Flight No.': [],
    'Launch site': [],
    'Payload': [],
    'Payload mass': [],
    'Orbit': [],
    'Customer': [],
    'Launch outcome': [],
    'Version Booster': [],
    'Booster landing': [],
    'Date': [],
    'Time': []
}

extracted_row = 0

for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):
    for rows in table.find_all("tr"):
        # Определяем, что это строка с номером полёта
        if rows.th and rows.th.string and rows.th.string.strip().isdigit():
            flight_number = rows.th.string.strip()
            flag = True
        else:
            flag = False

        # Список <td> в этой строке
        row = rows.find_all('td')

        if flag and len(row) >= 9:
            extracted_row += 1

            # 1) Flight No.
            launch_dict['Flight No.'].append(flight_number)

            # 2) Date и Time
            dt_parts = date_time(row[0])  # [date, time]
            launch_dict['Date'].append(dt_parts[0].rstrip(','))
            launch_dict['Time'].append(dt_parts[1])

            # 3) Version Booster
            bv = booster_version(row[1])
            if not bv and row[1].a:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)

            # 4) Launch site
            ls = row[2].a.string if row[2].a else row[2].get_text(strip=True)
            launch_dict['Launch site'].append(ls)

            # 5) Payload
            pl = row[3].a.string if row[3].a else row[3].get_text(strip=True)
            launch_dict['Payload'].append(pl)

            # 6) Payload mass
            pm = get_mass(row[4])
            launch_dict['Payload mass'].append(pm)

            # 7) Orbit
            orb = row[5].a.string if row[5].a else row[5].get_text(strip=True)
            launch_dict['Orbit'].append(orb)

            # 8) Customer
            cust = row[6].a.string if row[6].a else row[6].get_text(strip=True)
            launch_dict['Customer'].append(cust)

            # 9) Launch outcome
            lo = list(row[7].stripped_strings)[0]
            launch_dict['Launch outcome'].append(lo)

            # 10) Booster landing
            bl = landing_status(row[8])
            launch_dict['Booster landing'].append(bl)

# После цикла проверим количество собранных записей
print(f"Extracted rows: {extracted_row}")
# И создадим DataFrame
df_launch = pd.DataFrame(launch_dict)
df_launch.head()

            

Extracted rows: 121


,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [32]:
# Простой способ без депрецирующих Series:
df = pd.DataFrame.from_dict(launch_dict)

# Либо, если хотите явно задать dtype для пустых списков:
df = pd.DataFrame({
    key: pd.Series(value, dtype=object)
    for key, value in launch_dict.items()
})


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [33]:
# Экспортируем полученный DataFrame в CSV
df.to_csv('spacex_web_scraped.csv', index=False)
print("Данные успешно сохранены в файл spacex_web_scraped.csv")


Данные успешно сохранены в файл spacex_web_scraped.csv


In [35]:
import requests
import pandas as pd

# 1) Получаем все запуски из SpaceX API v4
launches = requests.get("https://api.spacexdata.com/v4/launches").json()
df = pd.json_normalize(launches)

# 1. Какой год в первой строке столбца static_fire_date_utc?
first_year = pd.to_datetime(df.loc[0, 'static_fire_date_utc']).year
print("1:", first_year)

# 2. Сколько запусков Falcon 9 после удаления Falcon 1?
# Сначала находим ID ракеты Falcon 1
rockets = requests.get("https://api.spacexdata.com/v4/rockets").json()
falcon1_id = next(r['id'] for r in rockets if r['name'] == "Falcon 1")

# Считаем запуски, у которых rocket != falcon1_id
falcon9_count = df[df['rocket'] != falcon1_id].shape[0]
print("2:", falcon9_count)

# 3. Сколько пропусков в колонке landing_pad?
missing = df['landing_pad'].isna().sum()
print("3:", missing)



1: 2006
2: 200


KeyError: 'landing_pad'

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
